In [ ]:
# Stream 1: Importation et préparation des données

In [8]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()


Saving BaseDonnee8.xlsx to BaseDonnee8 (2).xlsx


In [9]:
df = pd.read_excel('BaseDonnee8.xlsx')
df.replace("..", np.nan, inplace=True)


<ipython-input-9-809fe13f1ab8>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("..", np.nan, inplace=True)


In [10]:
econ_df = df[df['Series Name'] == 'GDP per capita (current US$)'][['Country Name','2019 [YR2019]','2020 [YR2020]']].copy()
econ_df['2019 [YR2019]'] = pd.to_numeric(econ_df['2019 [YR2019]'], errors='coerce')
econ_df['2020 [YR2020]'] = pd.to_numeric(econ_df['2020 [YR2020]'], errors='coerce')
econ_df['GDP_pct_2019'] = (econ_df['2019 [YR2019]'] - econ_df['2019 [YR2019]'].min())/(econ_df['2019 [YR2019]'].max()-econ_df['2019 [YR2019]'].min())
econ_df['GDP_pct_2020'] = (econ_df['2020 [YR2020]'] - econ_df['2020 [YR2020]'].min())/(econ_df['2020 [YR2020]'].max()-econ_df['2020 [YR2020]'].min())


In [13]:
social_df = df[df['Series Name'] == 'Literacy rate, adult total (% of people ages 15 and above)'][['Country Name','2019 [YR2019]','2020 [YR2020]']].copy()
social_df['2019 [YR2019]'] = pd.to_numeric(social_df['2019 [YR2019]'], errors='coerce')
social_df['2020 [YR2020]'] = pd.to_numeric(social_df['2020 [YR2020]'], errors='coerce')
social_df.rename(columns={'2019 [YR2019]':'Literacy_2019','2020 [YR2020]':'Literacy_2020'}, inplace=True)


In [14]:
env_df = df[df['Series Name'] == 'CO2 emissions (metric tons per capita)'][['Country Name','2019 [YR2019]','2020 [YR2020]']].copy()
env_df['2019 [YR2019]'] = pd.to_numeric(env_df['2019 [YR2019]'], errors='coerce')
env_df['2020 [YR2020]'] = pd.to_numeric(env_df['2020 [YR2020]'], errors='coerce')
env_df['CO2_pct_2019'] = (env_df['2019 [YR2019]'] - env_df['2019 [YR2019]'].min())/(env_df['2019 [YR2019]'].max()-env_df['2019 [YR2019]'].min())
env_df['CO2_pct_2020'] = (env_df['2020 [YR2020]'] - env_df['2020 [YR2020]'].min())/(env_df['2020 [YR2020]'].max()-env_df['2020 [YR2020]'].min())


In [15]:
data2019 = pd.merge(econ_df[['Country Name','GDP_pct_2019']], social_df[['Country Name','Literacy_2019']], on='Country Name')
data2019 = pd.merge(data2019, env_df[['Country Name','CO2_pct_2019']], on='Country Name')
data2020 = pd.merge(econ_df[['Country Name','GDP_pct_2020']], social_df[['Country Name','Literacy_2020']], on='Country Name')
data2020 = pd.merge(data2020, env_df[['Country Name','CO2_pct_2020']], on='Country Name')


In [16]:
data2019.rename(columns={'GDP_pct_2019':'IE 2019','Literacy_2019':'IS 2019','CO2_pct_2019':'IEV 2019'}, inplace=True)
data2020.rename(columns={'GDP_pct_2020':'IE 2020','Literacy_2020':'IS 2020','CO2_pct_2020':'IEV 2020'}, inplace=True)


In [17]:
data2019.head()


,Country Name,IE 2019,IS 2019,IEV 2019
0,Afghanistan,0.001428,NaN,0.003787
1,Albania,0.027329,NaN,0.051029
2,Algeria,0.019897,NaN,0.121485
3,American Samoa,0.060717,NaN,NaN
4,Andorra,0.214910,NaN,0.198666


In [18]:
data2020.head()


,Country Name,IE 2020,IS 2020,IEV 2020
0,Afghanistan,0.001631,NaN,NaN
1,Albania,0.029393,NaN,NaN
2,Algeria,0.017717,NaN,NaN
3,American Samoa,0.072705,NaN,NaN
4,Andorra,0.214367,NaN,NaN


In [ ]:
# Stream 2: Nettoyage et fusion des données dans un DataFrame global

In [20]:
data2019 = data2019.fillna(data2019.mean(numeric_only=True))
data2020 = data2020.fillna(data2020.mean(numeric_only=True))


In [21]:
data2019['ISD 2019'] = (data2019['IE 2019'] + data2019['IS 2019']) / 2
data2020['ISD 2020'] = (data2020['IE 2020'] + data2020['IS 2020']) / 2


In [22]:
globaldata = pd.merge(data2019[['Country Name','ISD 2019','IEV 2019']], data2020[['Country Name','ISD 2020','IEV 2020']], on='Country Name')
globaldata.rename(columns={'ISD 2019':'Indicateur socio-économique 2019','ISD 2020':'Indicateur socio-économique 2020','IEV 2019':'Indicateur environnemental 2019','IEV 2020':'Indicateur environnemental 2020'}, inplace=True)


In [23]:
globaldata.head()


,Country Name,Indicateur socio-économique 2019,Indicateur environnemental 2019,Indicateur socio-économique 2020,Indicateur environnemental 2020
0,Afghanistan,42.853278,0.003787,42.265066,NaN
1,Albania,42.866229,0.051029,42.278947,NaN
2,Algeria,42.862513,0.121485,42.273109,NaN
3,American Samoa,42.882923,0.126614,42.300603,NaN
4,Andorra,42.960019,0.198666,42.371434,NaN


In [ ]:
# Stream 3: Classification qualitative et taux d'évolution

In [26]:
def classify_series(series):
    min_val = series.min()
    max_val = series.max()
    bound1 = min_val + (max_val - min_val) / 3
    bound2 = min_val + 2 * (max_val - min_val) / 3
    conditions = [series < bound1, (series >= bound1) & (series < bound2), series >= bound2]
    choices = ['faible', 'moyen', 'eleve']
    return np.select(conditions, choices, default='unknown')


In [27]:
globaldata['ISEL-2019'] = classify_series(globaldata['Indicateur socio-économique 2019'])
globaldata['ISEL-2020'] = classify_series(globaldata['Indicateur socio-économique 2020'])
globaldata['IEVL-2019'] = classify_series(globaldata['Indicateur environnemental 2019'])
globaldata['IEVL-2020'] = classify_series(globaldata['Indicateur environnemental 2020'])


In [28]:
globaldata.head()

,Country Name,Indicateur socio-économique 2019,Indicateur environnemental 2019,Indicateur socio-économique 2020,Indicateur environnemental 2020,ISEL-2019,ISEL-2020,IEVL-2019,IEVL-2020
0,Afghanistan,42.853278,0.003787,42.265066,NaN,moyen,eleve,faible,unknown
1,Albania,42.866229,0.051029,42.278947,NaN,moyen,eleve,faible,unknown
2,Algeria,42.862513,0.121485,42.273109,NaN,moyen,eleve,faible,unknown
3,American Samoa,42.882923,0.126614,42.300603,NaN,moyen,eleve,faible,unknown
4,Andorra,42.960019,0.198666,42.371434,NaN,moyen,eleve,faible,unknown


In [29]:
ct_2019 = pd.crosstab(globaldata['ISEL-2019'], globaldata['IEVL-2019'])
ct_2020 = pd.crosstab(globaldata['ISEL-2020'], globaldata['IEVL-2020'])

In [30]:
ct_2019_flat = ct_2019.stack().reset_index(name='Count')
ct_2019_flat.sort_values(by='Count', ascending=False, inplace=True)
ct_2020_flat = ct_2020.stack().reset_index(name='Count')
ct_2020_flat.sort_values(by='Count', ascending=False, inplace=True)
ct_2019_flat, ct_2020_flat


(  ISEL-2019 IEVL-2019  Count
 7     moyen    faible    182
 1     eleve    faible     49
 8     moyen     moyen     12
 4    faible    faible     12
 2     eleve     moyen      4
 0     eleve     eleve      1
 6     moyen     eleve      1
 3    faible     eleve      0
 5    faible     moyen      0,
   ISEL-2020 IEVL-2020  Count
 0     eleve   unknown    243
 2     moyen   unknown     17
 1    faible   unknown      1)

In [31]:
def classify_oase(isel, ievl):
    if isel == 'faible' and ievl == 'faible':
        return "potentiel d'amélioration durable"
    elif isel == 'moyen' and ievl == 'faible':
        return "potentiel d'amélioration écologique avec stabilité socio-économique"
    elif isel == 'faible' and (ievl == 'moyen' or ievl == 'eleve'):
        return "potentiel d'amélioration socio-économique"
    elif isel == 'moyen' and ievl == 'moyen':
        return "potentiel d'amélioration socio-économique et écologique"
    elif isel == 'eleve' and ievl == 'eleve':
        return "potentiel fortement durable"
    else:
        return "autre"


In [32]:
globaldata['OASE 2019'] = globaldata.apply(lambda row: classify_oase(row['ISEL-2019'], row['IEVL-2019']), axis=1)
globaldata['OASE 2020'] = globaldata.apply(lambda row: classify_oase(row['ISEL-2020'], row['IEVL-2020']), axis=1)


In [33]:
globaldata['SEVL 2019'] = globaldata['ISEL-2019'] + "-" + globaldata['IEVL-2019']
globaldata['SEVL 2020'] = globaldata['ISEL-2020'] + "-" + globaldata['IEVL-2020']


In [34]:
globaldata['Socio_Economic_Evolution_Rate'] = (globaldata['Indicateur socio-économique 2020'] - globaldata['Indicateur socio-économique 2019']) / globaldata['Indicateur socio-économique 2019']
globaldata['Ecologic_Evolution_Rate'] = (globaldata['Indicateur environnemental 2020'] - globaldata['Indicateur environnemental 2019']) / globaldata['Indicateur environnemental 2019']


In [35]:
def classify_evolution(rate):
    if rate < 0:
        return "diminution"
    elif rate > 0.05:
        return "fort"
    else:
        return "moyen"


In [36]:
globaldata.head()


,Country Name,Indicateur socio-économique 2019,Indicateur environnemental 2019,Indicateur socio-économique 2020,Indicateur environnemental 2020,ISEL-2019,ISEL-2020,IEVL-2019,IEVL-2020,OASE 2019,OASE 2020,SEVL 2019,SEVL 2020,Socio_Economic_Evolution_Rate,Ecologic_Evolution_Rate
0,Afghanistan,42.853278,0.003787,42.265066,NaN,moyen,eleve,faible,unknown,potentiel d'amélioration écologique avec stabi...,autre,moyen-faible,eleve-unknown,-0.013726,NaN
1,Albania,42.866229,0.051029,42.278947,NaN,moyen,eleve,faible,unknown,potentiel d'amélioration écologique avec stabi...,autre,moyen-faible,eleve-unknown,-0.013700,NaN
2,Algeria,42.862513,0.121485,42.273109,NaN,moyen,eleve,faible,unknown,potentiel d'amélioration écologique avec stabi...,autre,moyen-faible,eleve-unknown,-0.013751,NaN
3,American Samoa,42.882923,0.126614,42.300603,NaN,moyen,eleve,faible,unknown,potentiel d'amélioration écologique avec stabi...,autre,moyen-faible,eleve-unknown,-0.013579,NaN
4,Andorra,42.960019,0.198666,42.371434,NaN,moyen,eleve,faible,unknown,potentiel d'amélioration écologique avec stabi...,autre,moyen-faible,eleve-unknown,-0.013701,NaN


In [ ]:
# Stream 4: Exportation des données finales pour la visualisation Power BI

In [37]:
globaldata.to_excel('globaldata_final.xlsx', index=False)


In [38]:
from google.colab import files
files.download('globaldata_final.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>